In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import random
import pickle
from itertools import repeat

def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale    


def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df["user"]==user_id]
    user_df=user_df.loc[user_df["episode_class"]!=-1]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/user_matrix_duration.pickle', 'rb') as handle:
            user_matrix_duration = pickle.load(handle)
            
    with open('/home/jupyter/sneupane/MOODS/pickled_files/user_matrix_stressor.pickle', 'rb') as handle:
        user_matrix_stressor = pickle.load(handle)
        
    with open('/home/jupyter/sneupane/MOODS/pickled_files/user_matrix_likelihood.pickle', 'rb') as handle:
        user_matrix_likelihood = pickle.load(handle)
        
    with open('/home/jupyter/sneupane/MOODS/pickled_files/user_matrix_location.pickle', 'rb') as handle:
        user_matrix_location = pickle.load(handle)
        
    with open('/home/jupyter/sneupane/MOODS/pickled_files/user_matrix_day_hour_minutes.pickle', 'rb') as handle:
        user_matrix_day_hour_minutes = pickle.load(handle)
    with open('/home/jupyter/sneupane/MOODS/pickled_files/stressor_abbr.pickle', 'rb') as handle:
            stressor_lemma = pickle.load(handle)
            
    y_modified_labels=stressor_lemma[user_id]
        
    if len(user_df )> 0:
        
        
       
        
        data_avaliable_hour=[]
 
        days=['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat','Sun']
        day=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
        duration=user_matrix_duration[user_id]
        for i in duration:
#             print(i)
            for j in i:
                if j !=0:
                    data_avaliable_hour.append(duration.index(i))
#         print(z)
        duration.reverse()
#         print(data_avaliable_hour)
        
                    
        
        hour_window=[]
        hour_window_full=[]
        for hour in range(data_avaliable_hour[0],24):
            hour_window.append(str(hour)+":00")
        for hour in range(0,24):
            hour_window_full.append(str(hour)+":00")
        hour_window.reverse()
     
        likelihood=user_matrix_likelihood[user_id]
        likelihood.reverse()
        stressors=user_matrix_stressor[user_id]
        stressors.reverse()
        locations=user_matrix_location[user_id]
        locations.reverse()

        total_duration=user_matrix_day_hour_minutes[user_id]
        total_duration.reverse()
        
        hoverdata= [[None,None,None,None,None,None,None] for i in repeat(None, 24)]
#         print(user_id)
        for n, row in enumerate(duration):
            for m, val in enumerate(row):
#                 print("Stressed : " + str(duration[n][m]))
                
#                 print("total : "+ str(total_duration[n][m]))
#                 print()
                if duration[n][m]!= None and stressors[n][m]!=None:
                    hoverdata[n][m]="<b> Day<b>: " + str(day[m])+"<br><b> Top Stressors<b>: " + str(stressors[n][m]) + \
                    "<br><b> Location<b>: " + str(locations[n][m])+ "<br><b> Stress Probability:  " + str(round(likelihood[n][m],2))[1:]+ "<br><b> Average Stressed minutes per hour: " + \
                    str(round(duration[n][m]/total_duration[n][m] * 60,2))
        bvals = [0,10,20,30,40,50,60,70,80,90,100]
        bvals=[i/100 for i in bvals ]
        
        colors=['#ff9868', '#ff7c51', '#ff5c37', '#ff2b16', '#ec0000', '#d00000', '#b50000', '#9b0000', '#820000', '#690000']


#         dcolorsc = discrete_colorscale(bvals, colors)


        import plotly.graph_objects as go
#         print(hour_window)
        fig2 = go.Figure(data=[go.Heatmap(z=likelihood,x=days,xgap =4,ygap=4,y=hour_window, 
                                        customdata=hoverdata,hoverinfo='text',hovertemplate="%{customdata}",hoverongaps = False,showscale=True,colorscale =colors,
                                          name="",zmin=0, zmax=1,colorbar = dict(thickness=7))])

#         print(hour_window)
        annotations = go.Annotations()
        for n, row in enumerate(likelihood):
            for m, val in enumerate(row):

#                 if n < len(hour_window):
                    if likelihood[n][m]!= None:
                        
                        annotations.append(go.Annotation(text=str(y_modified_labels[stressors[n][m]]) ,
                                                     x=days[m], y=hour_window[n],
                                                     xref='x1', yref='y1', showarrow=False,
                                                         font={"family":"Open Sans","size":10,"color":"Black"},
                                                       ))


#         fig.update_traces(textposition='top center')
        fig2.update_yaxes(nticks=48)
        fig2.update_xaxes(side="top",ticks="outside",mirror=True,
                  showline=True,linewidth=2,
            tickson="boundaries",
            ticklen=20)
        fig2.update_yaxes(
            side="top",
                          ticks="outside",mirror=True,
#                           tickvals=y,
                  type="category",
            showline=True,linewidth=1,
            tickson="boundaries",
            ticklen=20
        )
        fig2.update_layout(autosize=False,showlegend=True, template="gridon",annotations=annotations)
                          
        fig2.update_layout(legend=dict(orientation="h",
                                itemclick="toggleothers",
                                itemdoubleclick="toggle",  y=1.15,
    xanchor="right",
    x=1,font=dict(
            family="Courier",
            size=12,
            color="black"
        ),),
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
                font_family="Rockwell",
                bordercolor="Black"
            )
        )
#         print(y)
        # fig.update_xaxes(title_font_family="Arial",title_font_color="red")
        fig2.layout.xaxis.color = 'Black'
        fig2.layout.yaxis.color = 'gray'
        fig2.update_xaxes(fixedrange=True)
        fig2.update_yaxes(fixedrange=True)
        fig2.layout.xaxis.gridcolor="black"
        fig2.layout.xaxis.gridwidth=2
        fig2.layout.xaxis.tickfont={"family":"Times New Roman","size":15}
        fig2.layout.yaxis.gridcolor="black"
        fig2.layout.yaxis.gridwidth=2
#         fig2.update_yaxes(range=[10,22],type="category",categoryorder="array",categoryarray=hour_window)
#         fig2.update_traces(showlegend=True)
        fig2.update_layout(
                    autosize=True,
                        height=1400,
           
                    margin=dict(
                        l=40,
                        r=40,
                        b=10,
                        t=60,
#                         pad=20
                       
                    ),)
        data_summary_user=data_summary_user_current["Calendar View"]
#         print(data_summary_user)
        
        return "Calendar View",fig2,data_summary_user
    # fig.update_traces(showlegend=False)
    # fig2.show()
    # fig2.write_html("calendar.html")
    else:
        return "","",""